# GANomaly
- 2020.08.17 : gen 10 / lr 1e-2

# 개발일지
- 2020.08.15 : 분율 추출 구현 / 이상치 상관계수 추출 구현 / 모델 save, load 구현
- 2020.08.16 : anomaly detect 이미지 저장 구현 / 티타늄(Ti64) 상대 밀도 계산 구현

--- 

# Module

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
import os
import numpy as np

import torchvision.datasets as dataset
import torchvision.transforms as transforms
import torchvision.utils as vutils

# Graph & Animation
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Hyper Parameter

In [2]:
latent_size = 100  # z벡터의 잠재공간(latent space)의 크기
workers = 4    # 0일때, 약 20% 사용 4일 경우 메모리 100%
img_size = 64
channel = 1
epochs = 100
batch_size = 64
learning_rate = 1e-2

# Data and Device

In [3]:
# 이미지 경로, 이미지들 리사이즈 및 텐서형태로 만들기
#data_root = "../../dataset/train"
Ti64_aug_data_root = "../../dataset/aug_train/aug_Ti64"

data_set = dataset.ImageFolder(root = Ti64_aug_data_root, # data_root,
                           transform = transforms.Compose([
                                  #transforms.Resize(img_size),
                                  transforms.CenterCrop(img_size),
                                  torchvision.transforms.Grayscale(channel),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5,),(0.5,))
                              ]))

print("size of dataset :", len(data_set))

# 배치로 나누고 셔플하기
data_loader = torch.utils.data.DataLoader(data_set, batch_size = batch_size,
                                         shuffle = True, num_workers = workers, drop_last=True)

# Device setting (GPU or CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device :", device)

size of dataset : 1020
device : cuda:0


# MODEL

In [4]:
# GANomaly : Generative Adversarial Networks Model using Decoder with AutoEncoder

# === Decoder 모델 ===
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        
        # Batch Normalization 유무에 따른 G_block 정의
        def DC_conv_block(in_features, out_features):
            block = [
                    nn.Conv2d(in_features, out_features, 3, 1, 1, bias=False),
                    nn.BatchNorm2d(out_features),
                    nn.ELU()
            ]
            return block
        
        def DC_deconv_block(in_features, out_features, FIRST=True):
            if FIRST:
                block = [
                    nn.ConvTranspose2d(in_features, out_features, 4, 1, 0, bias=False),
                    nn.BatchNorm2d(out_features),
                    nn.ELU()
                ]
            else:
                block = [
                    nn.ConvTranspose2d(in_features, out_features, 4, 2, 1, bias=False),
                    nn.BatchNorm2d(out_features),
                    nn.ELU()
                ]
            return block
        
        
        # ======================= 픽셀 분포 생성 layer ======================= 
        self.DC_gen_distribution = nn.Sequential(
            # ------ input is latent_size 100 ------ 
            *DC_deconv_block(latent_size, img_size*8, FIRST=True),
            # ------ state size is 512x4x4 ------ 
            *DC_conv_block(img_size*8, img_size*8),
            # ------ state size is 512x4x4 ------   
            *DC_deconv_block(img_size*8, img_size*4, FIRST=False),
            # ------ state size is 256x8x8 ------ 
            *DC_conv_block(img_size*4, img_size*4),
            # ------ state size is 256x8x8 ------ 
            *DC_deconv_block(img_size*4, img_size*2, FIRST=False),
            # ------ state size is 128x16x16 ------ 
            *DC_conv_block(img_size*2, img_size*2),
            # ------ state size is 128x16x16 ------ 
            *DC_deconv_block(img_size*2, img_size, FIRST=False),
        )
        
        # =================== 가짜 이미지 생성 layer =================== 
        self.DC_gen_fake_img = nn.Sequential(
            # ------ state size is 64x32x32 ------ 
            nn.ConvTranspose2d(img_size, 1, 4, 2, 1, bias=False),
            nn.Tanh() # 픽셀값의 범위 : -1 ~ 1로 두기 위해서
            # ------ state size is 1x64x64 ------ 
        )
    
    def forward(self, input):
        distribution = self.DC_gen_distribution(input)
        fake_img = self.DC_gen_fake_img(distribution)
        
        return fake_img


# === Discriminator 모델 ===
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        # Batch Normalization 유무에 따른 D_block 정의
        def D_block(in_features, out_features, MP=True):
            if MP:
                block = [
                    nn.Conv2d(in_features, out_features, 3, 1, 1, bias=False),
                    nn.BatchNorm2d(out_features),
                    nn.ELU()
                ]
            else:
                block = [
                    nn.Conv2d(in_features, out_features, 3, 1, 1, bias=False),
                    nn.BatchNorm2d(out_features),
                    nn.ELU(),
                    nn.MaxPool2d(2)
                ]
            return block
        
        # ============== Feature 추출 layer ==============
        self.D_extract_feature = nn.Sequential(
            # ------ input is 1 x 64 x 64 ------ 
            *D_block(channel, img_size, MP=False),
            # ------ input is 64 x 64 x 64 ------ 
            *D_block(img_size, img_size, MP=True),
            # ------ state is 64 x 32 x 32 ------ 
            *D_block(img_size, img_size*2, MP=False),
            # ------ state is 128 x 32 x 32 ------ 
            *D_block(img_size*2, img_size*2, MP=True),
            # ------ state is 128 x 16 x 16 ------ 
            *D_block(img_size*2, img_size*4, MP=False),
            # ------ state is 256 x 16 x 16 ------ 
            *D_block(img_size*4, img_size*4, MP=True),
            # ------ state is 256 x 8 x 8 ------ 
            *D_block(img_size*4, img_size*8, MP=False),
            # ------ state is 512 x 8 x 8 ------ 
            *D_block(img_size*8, img_size*8, MP=True),
        )
        
        # ===================== 이진 분류 layer =====================
        self.D_classification = nn.Sequential(        
            # ------- state size 512x4x4 ------- 
            nn.Conv2d(img_size*8, channel, 4, 1, 0, bias=False),
            #nn.Linear(fms*8*4*4, 1, bias=False),
            nn.Sigmoid()        
        )
        
    def forward(self, input):
        feature = self.D_extract_feature(input)
        classification = self.D_classification(feature)
        
        return classification, feature 


# === Encoder Model ===
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        def EC_block(in_features, out_features, MP=True):
            if MP:
                block = [
                    nn.Conv2d(in_features, out_features, 3, 1, 1, bias=False),
                    nn.BatchNorm2d(out_features),
                    nn.ELU()
                ]
            else:
                block = [
                    nn.Conv2d(in_features, out_features, 3, 1, 1, bias=False),
                    nn.BatchNorm2d(out_features),
                    nn.ELU(),
                    nn.MaxPool2d(2)
                ]
            return block
        
        # ============== Feature 추출 ==============
        self.EC_extract_feature = nn.Sequential(
            # ------ input is 1 x 64 x 64 ------ 
            *EC_block(channel, img_size, MP=False),
            # ------ input is 64 x 64 x 64 ------ 
            *EC_block(img_size, img_size, MP=True),
            # ------ state is 64 x 32 x 32 ------ 
            *EC_block(img_size, img_size*2, MP=False),
            # ------ state is 128 x 32 x 32 ------ 
            *EC_block(img_size*2, img_size*2, MP=True),
            # ------ state is 128 x 16 x 16 ------ 
            *EC_block(img_size*2, img_size*4, MP=False),
            # ------ state is 256 x 16 x 16 ------ 
            *EC_block(img_size*4, img_size*4, MP=True),
            # ------ state is 256 x 8 x 8 ------ 
            *EC_block(img_size*4, img_size*8, MP=False),
            # ------ state is 512 x 8 x 8 ------ 
            *EC_block(img_size*8, img_size*8, MP=True),
        )
        
        # =============== Encoder Training layer ===============
        self.EC_validate = nn.Sequential(
            # -------state is 512 x 4 x 4-------
            nn.Conv2d(img_size*8, latent_size, 4, 1, 0, bias=False),
            nn.Tanh()
            # -------state is 100 x 97 x 97-------
        )
        
    def forward(self, input):
        feature = self.EC_extract_feature(input)
        validity = self.EC_validate(feature)
        
        return validity

# Loss & Optim

In [5]:
# DC and D 무게 초기화, classname 에 찾는 name가 없다면 -1 ,
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


# D,DC,EC 네트워크 모델 객체 선언
D = Discriminator().to(device)
DC = Decoder().to(device)
EC = Encoder().to(device)

# weight initialize/ nn.Module 클래스 안에 apply 함수가 정의되 있음, 각 함수들에 다 적용 하게한다
D.apply(weights_init)
DC.apply(weights_init)
EC.apply(weights_init)

# Binary cross entropy loss and optimizer
DCGAN_criterion = nn.BCELoss()
AE_criterion = nn.MSELoss()

# latent vector에 배치 사이즈 64를 적용
# 학습한 DC로 새로운 것 만들어서 결과 확인 할때 사용
noise_z = torch.randn(img_size, latent_size, 1, 1, device = device)

# D와 DC에 대해 두가지 최적화 설정
D_optimizer = torch.optim.Adam(D.parameters(), lr = learning_rate, betas=(0.5,0.999))
DC_optimizer = torch.optim.Adam(DC.parameters(), lr = learning_rate, betas=(0.5,0.999))
EC_optimizer = torch.optim.Adam(EC.parameters(), lr = learning_rate, betas=(0.5,0.999))

#print(D)
#print(DC)
#print(EC)

def reset_grad():
    D_optimizer.zero_grad()
    DC_optimizer.zero_grad()

---

# Train DCGAN

In [6]:
img_list = []
DC_losses = []
D_losses = []
iters = 0

def train_DCGAN(epoch, learning_DC_per_D = 4):
    global epochs
    global iters
    
    # 인덱스 0부터 세기 시작
    # data[0].size():64x1x64x64(image) / data[1].size():64(label)
    for i,data in enumerate(data_loader,0):
        
        # Train D
        real_img = data[0].to(device) # image size: 64x1x64x64(batch, channel, width, height)
        b_size = real_img.size(0) # b_size = 64
        real_labels = torch.ones(b_size, 1).to(device)
        fake_labels = torch.zeros(b_size, 1).to(device)
        
        # (--------------------------real-----------------------------)
        real_classification, _ = D(real_img) # output = D(x)
        real_loss = DCGAN_criterion(real_classification, real_labels) # D(x)=1일 때의 loss
        real_score = real_classification
        D_x = real_score.mean().item() 
            
        # (--------------------------fake-----------------------------)
        z = torch.randn(b_size, latent_size, 1, 1).to(device) # z size :64x100x1x1
        fake_img = DC(z)
        fake_classification, _ = D(fake_img) # output = D(DC(z))
        fake_loss = DCGAN_criterion(fake_classification, fake_labels) # D(DC(z))=0일 때의 loss
        fake_score = fake_classification
        D_DC_z1 = fake_score.mean().item()

        # (------------------Backprop and optimize---------------------)
        D_loss = real_loss + fake_loss
        reset_grad()
        D_loss.backward()
        D_optimizer.step() # D(x)=1, D(DC(z))=0이어야 D가 최적

        
        # Train DC
        #z = torch.randn(b_size,latent_size,1,1,device=device) # z size :64x100x1x1
        for k in range(learning_DC_per_D):
            fake_img = DC(z)
            fake_classification,_ = D(fake_img)  # output : D(DC(z))
            D_DC_z2 = fake_classification.mean().item()
            DC_loss = DCGAN_criterion(fake_classification, real_labels) # D(DC(z))=1일 때의 loss=log(D(DC(z)))

            # (------------------Backprop and optimize---------------------)
            reset_grad()
            DC_loss.backward()
            DC_optimizer.step() # D(DC(z))=1 이어야 DC가 최적
            # ==> D(DC(z))의 값이 0.5로 수렴해야 한다.
        
        
        # print
        print('[%d/%d][%d/%d]\n- D_loss : %.4f / DC_loss : %.4f\n- D(x):%.4f / D(DC(z1)) : %.4f / D(DC(z2)) : %.4f' 
                   %(epoch+1, epochs, i, len(data_loader),D_loss.item(),
                     DC_loss.item(),D_x,D_DC_z1,D_DC_z2))
        
        # Save Losses for plotting later
        DC_losses.append(DC_loss.item())
        D_losses.append(D_loss.item())
        
        #Check how the generator is doing by saving G's output on noise_z
        if (iters % 500 == 0) or ((epoch == epochs-1) and (i == len(data_loader)-1)):
            with torch.no_grad():
                fake = DC(noise_z).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1
        
#torch.save(E.state_dict(), 'E.ckpt')

In [7]:
for epoch in range(epochs):
    train_DCGAN(epoch, learning_DC_per_D = 10)

/home/piai/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 1, 1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


[1/100][0/15]
- D_loss : 1.3910 / DC_loss : 0.3009
- D(x):0.4948 / D(DC(z1)) : 0.4970 / D(DC(z2)) : 0.7401
[1/100][1/15]
- D_loss : 2.1665 / DC_loss : 0.6955
- D(x):0.4622 / D(DC(z1)) : 0.7416 / D(DC(z2)) : 0.4994
[1/100][2/15]
- D_loss : 1.6484 / DC_loss : 0.5869
- D(x):0.3735 / D(DC(z1)) : 0.4787 / D(DC(z2)) : 0.5561
[1/100][3/15]
- D_loss : 1.4300 / DC_loss : 0.5334
- D(x):0.5384 / D(DC(z1)) : 0.5553 / D(DC(z2)) : 0.5866
[1/100][4/15]
- D_loss : 1.6405 / DC_loss : 0.8121
- D(x):0.4853 / D(DC(z1)) : 0.5999 / D(DC(z2)) : 0.4440
[1/100][5/15]
- D_loss : 1.4426 / DC_loss : 0.6288
- D(x):0.4261 / D(DC(z1)) : 0.4445 / D(DC(z2)) : 0.5332
[1/100][6/15]
- D_loss : 1.4832 / DC_loss : 0.6899
- D(x):0.4867 / D(DC(z1)) : 0.5338 / D(DC(z2)) : 0.5016
[1/100][7/15]
- D_loss : 1.3925 / DC_loss : 0.6896
- D(x):0.4992 / D(DC(z1)) : 0.5021 / D(DC(z2)) : 0.5018
[1/100][8/15]
- D_loss : 1.4060 / DC_loss : 0.7259
- D(x):0.4919 / D(DC(z1)) : 0.5016 / D(DC(z2)) : 0.4842
[1/100][9/15]
- D_loss : 1.3992 / DC_

[6/100][2/15]
- D_loss : 1.3909 / DC_loss : 0.6886
- D(x):0.4940 / D(DC(z1)) : 0.4960 / D(DC(z2)) : 0.5023
[6/100][3/15]
- D_loss : 1.3894 / DC_loss : 0.6908
- D(x):0.5005 / D(DC(z1)) : 0.5020 / D(DC(z2)) : 0.5012
[6/100][4/15]
- D_loss : 1.3883 / DC_loss : 0.7009
- D(x):0.4995 / D(DC(z1)) : 0.5005 / D(DC(z2)) : 0.4961
[6/100][5/15]
- D_loss : 1.3888 / DC_loss : 0.6880
- D(x):0.4939 / D(DC(z1)) : 0.4951 / D(DC(z2)) : 0.5026
[6/100][6/15]
- D_loss : 1.3900 / DC_loss : 0.6788
- D(x):0.4982 / D(DC(z1)) : 0.5001 / D(DC(z2)) : 0.5072
[6/100][7/15]
- D_loss : 1.3897 / DC_loss : 0.6862
- D(x):0.5048 / D(DC(z1)) : 0.5064 / D(DC(z2)) : 0.5035
[6/100][8/15]
- D_loss : 1.3872 / DC_loss : 0.6938
- D(x):0.5020 / D(DC(z1)) : 0.5024 / D(DC(z2)) : 0.4997
[6/100][9/15]
- D_loss : 1.3880 / DC_loss : 0.6907
- D(x):0.4982 / D(DC(z1)) : 0.4991 / D(DC(z2)) : 0.5012
[6/100][10/15]
- D_loss : 1.3871 / DC_loss : 0.6921
- D(x):0.5005 / D(DC(z1)) : 0.5008 / D(DC(z2)) : 0.5006
[6/100][11/15]
- D_loss : 1.3893 / D

[11/100][4/15]
- D_loss : 1.2533 / DC_loss : 0.8640
- D(x):0.4919 / D(DC(z1)) : 0.4156 / D(DC(z2)) : 0.4231
[11/100][5/15]
- D_loss : 1.2294 / DC_loss : 0.8116
- D(x):0.4931 / D(DC(z1)) : 0.4057 / D(DC(z2)) : 0.4468
[11/100][6/15]
- D_loss : 1.2428 / DC_loss : 0.8252
- D(x):0.5020 / D(DC(z1)) : 0.4201 / D(DC(z2)) : 0.4394
[11/100][7/15]
- D_loss : 1.2701 / DC_loss : 0.7433
- D(x):0.5385 / D(DC(z1)) : 0.4756 / D(DC(z2)) : 0.4764
[11/100][8/15]
- D_loss : 1.5187 / DC_loss : 0.6078
- D(x):0.4298 / D(DC(z1)) : 0.4894 / D(DC(z2)) : 0.5450
[11/100][9/15]
- D_loss : 1.3092 / DC_loss : 0.7186
- D(x):0.5663 / D(DC(z1)) : 0.5214 / D(DC(z2)) : 0.4877
[11/100][10/15]
- D_loss : 1.2781 / DC_loss : 0.9553
- D(x):0.5207 / D(DC(z1)) : 0.4640 / D(DC(z2)) : 0.3848
[11/100][11/15]
- D_loss : 1.2686 / DC_loss : 0.6004
- D(x):0.4274 / D(DC(z1)) : 0.3411 / D(DC(z2)) : 0.5497
[11/100][12/15]
- D_loss : 1.3748 / DC_loss : 0.8031
- D(x):0.5807 / D(DC(z1)) : 0.5617 / D(DC(z2)) : 0.4518
[11/100][13/15]
- D_loss 

[16/100][5/15]
- D_loss : 0.0025 / DC_loss : 8.7934
- D(x):0.9981 / D(DC(z1)) : 0.0006 / D(DC(z2)) : 0.0007
[16/100][6/15]
- D_loss : 0.0019 / DC_loss : 8.8892
- D(x):0.9987 / D(DC(z1)) : 0.0006 / D(DC(z2)) : 0.0006
[16/100][7/15]
- D_loss : 0.0037 / DC_loss : 8.9665
- D(x):0.9970 / D(DC(z1)) : 0.0006 / D(DC(z2)) : 0.0005
[16/100][8/15]
- D_loss : 0.0020 / DC_loss : 8.9925
- D(x):0.9984 / D(DC(z1)) : 0.0005 / D(DC(z2)) : 0.0005
[16/100][9/15]
- D_loss : 0.0021 / DC_loss : 9.0602
- D(x):0.9983 / D(DC(z1)) : 0.0004 / D(DC(z2)) : 0.0005
[16/100][10/15]
- D_loss : 0.0015 / DC_loss : 9.1271
- D(x):0.9990 / D(DC(z1)) : 0.0005 / D(DC(z2)) : 0.0005
[16/100][11/15]
- D_loss : 0.0015 / DC_loss : 9.2263
- D(x):0.9989 / D(DC(z1)) : 0.0004 / D(DC(z2)) : 0.0004
[16/100][12/15]
- D_loss : 0.0015 / DC_loss : 9.2888
- D(x):0.9988 / D(DC(z1)) : 0.0004 / D(DC(z2)) : 0.0004
[16/100][13/15]
- D_loss : 0.0013 / DC_loss : 9.3596
- D(x):0.9991 / D(DC(z1)) : 0.0003 / D(DC(z2)) : 0.0003
[16/100][14/15]
- D_loss

...

[76/100][6/15]
- D_loss : 0.0000 / DC_loss : 13.4059
- D(x):1.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[76/100][7/15]
- D_loss : 0.0000 / DC_loss : 13.4046
- D(x):1.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[76/100][8/15]
- D_loss : 0.0000 / DC_loss : 13.4081
- D(x):1.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[76/100][9/15]
- D_loss : 0.0000 / DC_loss : 13.4132
- D(x):1.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[76/100][10/15]
- D_loss : 0.0000 / DC_loss : 13.4194
- D(x):1.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[76/100][11/15]
- D_loss : 0.0000 / DC_loss : 13.4107
- D(x):1.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[76/100][12/15]
- D_loss : 0.0000 / DC_loss : 13.4149
- D(x):1.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[76/100][13/15]
- D_loss : 0.0000 / DC_loss : 13.4182
- D(x):1.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[76/100][14/15]
- D_loss : 0.0000 / DC_loss : 13.4183
- D(x):1.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[77/100][0/15]

[81/100][6/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[81/100][7/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[81/100][8/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[81/100][9/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[81/100][10/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[81/100][11/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[81/100][12/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[81/100][13/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[81/100][14/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[82/1

[86/100][6/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[86/100][7/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[86/100][8/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[86/100][9/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[86/100][10/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[86/100][11/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[86/100][12/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[86/100][13/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[86/100][14/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[87/1

[91/100][6/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[91/100][7/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[91/100][8/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[91/100][9/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[91/100][10/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[91/100][11/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[91/100][12/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[91/100][13/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[91/100][14/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[92/1

[96/100][6/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[96/100][7/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[96/100][8/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[96/100][9/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[96/100][10/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[96/100][11/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[96/100][12/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[96/100][13/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[96/100][14/15]
- D_loss : 27.6310 / DC_loss : 27.6310
- D(x):0.0000 / D(DC(z1)) : 0.0000 / D(DC(z2)) : 0.0000
[97/1

# Animation

In [8]:
def animate_gen_imgs():
    fig = plt.figure(figsize=(8,8))
    plt.axis("off")
    
    ims=[[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
    ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)
    HTML(ani.to_jshtml())

In [9]:
animate_gen_imgs()

# Trian : izif E

In [10]:
DC.eval()
D.eval()

kappa = 1.0
EC_losses = []

iters = 0
AE_img_list = []

def train_AE(epoch):
    global epochs
    global iters
    global EC_losses
    global AE_img_list
    
    # 인덱스 0부터 세기 시작
    # data[0].size():64x1x64x64(image) / data[1].size():64(label)
    for i,data in enumerate(data_loader,0):
        
        real_imgs = data[0].to(device)
        
        EC_optimizer.zero_grad()
        
        EC_validity = EC(real_imgs)
        
        fake_imgs = DC(EC_validity)
        
        _, real_features = D.forward(real_imgs)
        _, fake_features = D.forward(fake_imgs)
        
        
        # izif architecture
        imgs_loss = AE_criterion(real_imgs, fake_imgs)
        features_loss = AE_criterion(real_features, fake_features)
        EC_loss = imgs_loss + kappa*features_loss
        
        EC_loss.backward()
        EC_optimizer.step()
        
        # =============================================================
        # print
        # =============================================================
        print('[%d/%d][%d/%d]\n- E_loss: %.4f\n'
              %(epoch+1, epochs, i, len(data_loader), EC_loss.item()))
        
        # Save Losses for plotting later
        EC_losses.append(EC_loss.item())
        
        #Check how the generator is doing by saving G's output on noise_z
        if (iters % 500 == 0) or ((epoch == epochs-1) and (i == len(data_loader)-1)):
            with torch.no_grad():
                fake = DC(EC_validity).detach().cpu()
            AE_img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1
        
#torch.save(E.state_dict(), 'E.ckpt')

In [ ]:
for epoch in range(epochs):
    train_AE(epoch)

[1/100][0/15]
- E_loss: 1.0283

[1/100][1/15]
- E_loss: 0.8559

[1/100][2/15]
- E_loss: 1.4775

[1/100][3/15]
- E_loss: 1.0208

[1/100][4/15]
- E_loss: 1.2541

[1/100][5/15]
- E_loss: 1.0438

[1/100][6/15]
- E_loss: 0.9355

[1/100][7/15]
- E_loss: 0.7883

[1/100][8/15]
- E_loss: 0.8192

[1/100][9/15]
- E_loss: 0.8051

[1/100][10/15]
- E_loss: 0.8929

[1/100][11/15]
- E_loss: 0.9011

[1/100][12/15]
- E_loss: 0.7877

[1/100][13/15]
- E_loss: 0.7996

[1/100][14/15]
- E_loss: 0.8175

[2/100][0/15]
- E_loss: 0.7529

[2/100][1/15]
- E_loss: 0.8242

[2/100][2/15]
- E_loss: 0.8271

[2/100][3/15]
- E_loss: 0.7396

[2/100][4/15]
- E_loss: 0.9000

[2/100][5/15]
- E_loss: 0.8348

[2/100][6/15]
- E_loss: 0.7213

[2/100][7/15]
- E_loss: 0.7725

[2/100][8/15]
- E_loss: 0.7957

[2/100][9/15]
- E_loss: 0.8803

[2/100][10/15]
- E_loss: 0.8955

[2/100][11/15]
- E_loss: 0.8810

[2/100][12/15]
- E_loss: 0.8552

[2/100][13/15]
- E_loss: 0.8211

[2/100][14/15]
- E_loss: 0.7314

[3/100][0/15]
- E_loss: 0.7859

[17/100][12/15]
- E_loss: 0.8681

[17/100][13/15]
- E_loss: 0.7465

[17/100][14/15]
- E_loss: 0.8293

[18/100][0/15]
- E_loss: 0.7709

[18/100][1/15]
- E_loss: 0.8356

[18/100][2/15]
- E_loss: 0.8687

[18/100][3/15]
- E_loss: 0.7127

[18/100][4/15]
- E_loss: 0.8805

[18/100][5/15]
- E_loss: 0.8757

[18/100][6/15]
- E_loss: 0.8015

[18/100][7/15]
- E_loss: 0.8574

[18/100][8/15]
- E_loss: 0.8309

[18/100][9/15]
- E_loss: 0.7784

[18/100][10/15]
- E_loss: 0.7806

[18/100][11/15]
- E_loss: 0.7677

[18/100][12/15]
- E_loss: 0.7868

[18/100][13/15]
- E_loss: 0.8036

[18/100][14/15]
- E_loss: 0.7588

[19/100][0/15]
- E_loss: 0.7844

[19/100][1/15]
- E_loss: 0.8431

[19/100][2/15]
- E_loss: 0.9121

[19/100][3/15]
- E_loss: 0.7634

[19/100][4/15]
- E_loss: 0.7673

[19/100][5/15]
- E_loss: 0.8044

[19/100][6/15]
- E_loss: 0.7640

[19/100][7/15]
- E_loss: 0.8859

[19/100][8/15]
- E_loss: 0.7980

[19/100][9/15]
- E_loss: 0.7938

[19/100][10/15]
- E_loss: 0.8591

[19/100][11/15]
- E_loss: 0.8678



[34/100][3/15]
- E_loss: 0.7812

[34/100][4/15]
- E_loss: 0.7305

[34/100][5/15]
- E_loss: 0.7334

[34/100][6/15]
- E_loss: 0.8800

[34/100][7/15]
- E_loss: 0.8917

[34/100][8/15]
- E_loss: 0.8699

[34/100][9/15]
- E_loss: 0.7866

[34/100][10/15]
- E_loss: 0.7965

[34/100][11/15]
- E_loss: 0.7781

[34/100][12/15]
- E_loss: 0.8319

[34/100][13/15]
- E_loss: 0.7861

[34/100][14/15]
- E_loss: 0.7589

[35/100][0/15]
- E_loss: 0.8144

[35/100][1/15]
- E_loss: 0.7679

[35/100][2/15]
- E_loss: 0.8522

[35/100][3/15]
- E_loss: 0.8582

[35/100][4/15]
- E_loss: 0.8320

[35/100][5/15]
- E_loss: 0.7618

[35/100][6/15]
- E_loss: 0.7648

[35/100][7/15]
- E_loss: 0.7348

[35/100][8/15]
- E_loss: 0.8033

[35/100][9/15]
- E_loss: 0.8285

[35/100][10/15]
- E_loss: 0.9203

[35/100][11/15]
- E_loss: 0.7896

[35/100][12/15]
- E_loss: 0.8050

[35/100][13/15]
- E_loss: 0.8204

[35/100][14/15]
- E_loss: 0.7737

[36/100][0/15]
- E_loss: 0.7809

[36/100][1/15]
- E_loss: 0.8119

[36/100][2/15]
- E_loss: 0.8294



[50/100][9/15]
- E_loss: 0.8421

[50/100][10/15]
- E_loss: 0.7518

[50/100][11/15]
- E_loss: 0.7881

[50/100][12/15]
- E_loss: 0.7990

[50/100][13/15]
- E_loss: 0.8419

[50/100][14/15]
- E_loss: 0.8647

[51/100][0/15]
- E_loss: 0.7848

[51/100][1/15]
- E_loss: 0.7589

[51/100][2/15]
- E_loss: 0.8310

[51/100][3/15]
- E_loss: 0.9058

[51/100][4/15]
- E_loss: 0.8249

[51/100][5/15]
- E_loss: 0.8318

[51/100][6/15]
- E_loss: 0.6875

[51/100][7/15]
- E_loss: 0.8096

[51/100][8/15]
- E_loss: 0.7985

[51/100][9/15]
- E_loss: 0.8536

[51/100][10/15]
- E_loss: 0.8102

[51/100][11/15]
- E_loss: 0.7529

[51/100][12/15]
- E_loss: 0.8795

[51/100][13/15]
- E_loss: 0.7667

[51/100][14/15]
- E_loss: 0.8077

[52/100][0/15]
- E_loss: 0.7593

[52/100][1/15]
- E_loss: 0.8151

[52/100][2/15]
- E_loss: 0.8267

[52/100][3/15]
- E_loss: 0.7953

[52/100][4/15]
- E_loss: 0.8021

[52/100][5/15]
- E_loss: 0.8446

[52/100][6/15]
- E_loss: 0.8021

[52/100][7/15]
- E_loss: 0.7875

[52/100][8/15]
- E_loss: 0.8075



[67/100][0/15]
- E_loss: 0.7375

[67/100][1/15]
- E_loss: 0.8450

[67/100][2/15]
- E_loss: 0.8116

[67/100][3/15]
- E_loss: 0.7481

[67/100][4/15]
- E_loss: 0.7796

[67/100][5/15]
- E_loss: 0.8137

[67/100][6/15]
- E_loss: 0.7575

[67/100][7/15]
- E_loss: 0.8033

[67/100][8/15]
- E_loss: 0.8196

[67/100][9/15]
- E_loss: 0.7976

[67/100][10/15]
- E_loss: 0.7680

[67/100][11/15]
- E_loss: 0.8135

[67/100][12/15]
- E_loss: 0.7658

[67/100][13/15]
- E_loss: 0.8362

[67/100][14/15]
- E_loss: 0.8725

[68/100][0/15]
- E_loss: 0.7570

[68/100][1/15]
- E_loss: 0.8182

[68/100][2/15]
- E_loss: 0.8063

[68/100][3/15]
- E_loss: 0.8081

[68/100][4/15]
- E_loss: 0.7848

[68/100][5/15]
- E_loss: 0.7164

[68/100][6/15]
- E_loss: 0.7803

[68/100][7/15]
- E_loss: 0.7891

[68/100][8/15]
- E_loss: 0.7523

[68/100][9/15]
- E_loss: 0.7831

[68/100][10/15]
- E_loss: 0.7868

[68/100][11/15]
- E_loss: 0.7544

[68/100][12/15]
- E_loss: 0.8466

[68/100][13/15]
- E_loss: 0.8165

[68/100][14/15]
- E_loss: 0.8458



[83/100][6/15]
- E_loss: 0.7680

[83/100][7/15]
- E_loss: 0.8128

[83/100][8/15]
- E_loss: 0.7697

[83/100][9/15]
- E_loss: 0.8088

[83/100][10/15]
- E_loss: 0.8214

[83/100][11/15]
- E_loss: 0.8077

[83/100][12/15]
- E_loss: 0.7851

[83/100][13/15]
- E_loss: 0.7813

[83/100][14/15]
- E_loss: 0.7354

[84/100][0/15]
- E_loss: 0.7832

[84/100][1/15]
- E_loss: 0.7667

[84/100][2/15]
- E_loss: 0.7786

[84/100][3/15]
- E_loss: 0.7858

[84/100][4/15]
- E_loss: 0.9077

[84/100][5/15]
- E_loss: 0.8550

[84/100][6/15]
- E_loss: 0.8202

[84/100][7/15]
- E_loss: 0.7139

[84/100][8/15]
- E_loss: 0.8156

[84/100][9/15]
- E_loss: 0.8005

[84/100][10/15]
- E_loss: 0.7881

[84/100][11/15]
- E_loss: 0.7840

[84/100][12/15]
- E_loss: 0.7683

[84/100][13/15]
- E_loss: 0.7940

[84/100][14/15]
- E_loss: 0.8379

[85/100][0/15]
- E_loss: 0.7882

[85/100][1/15]
- E_loss: 0.8924

[85/100][2/15]
- E_loss: 0.7814

[85/100][3/15]
- E_loss: 0.8067

[85/100][4/15]
- E_loss: 0.7687

[85/100][5/15]
- E_loss: 0.7704



# Loss Graph

In [ ]:
def plot_train_loss():
    plt.figure(figsize=(10,5))
    plt.title("Generator and Discriminator Loss During Training")
    plt.plot(DC_losses, label="G")
    plt.plot(D_losses, label="D")
    plt.plot(EC_losses, label="E")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [ ]:
plot_train_loss()

---

# Test Data

In [ ]:
#test_data_root = "../../dataset/test/test_DualPhaseSteel"
Ti64_test_data_root = "../../dataset/test/test_Ti64"

test_data_set = dataset.ImageFolder(root = Ti64_test_data_root, # test_data_root,
                           transform = transforms.Compose([
                                  transforms.Resize(img_size),
                                  transforms.CenterCrop(img_size),
                                  torchvision.transforms.Grayscale(channel),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5,),(0.5,))
                              ]))

# 배치로 나누고 셔플하기
test_data_loader = torch.utils.data.DataLoader(test_data_set, batch_size = 1,
                                              shuffle = False, num_workers = workers)

# 이상 픽셀 수 확인 함수

In [ ]:
def count_el_not_0(diff_img):
    count_el_not_0 = 0
    
    col_size = diff_img.shape[0]
    row_size = diff_img.shape[1]
    
    #print(col_size, row_size)
    
    for col in range(col_size):
        for row in range(row_size):
            if diff_img[col][row] != 0:
                count_el_not_0 += 1
                
    return count_el_not_0

# img 비교 함수

In [ ]:
def compare_imgs(real_img, generated_img, i, score, reverse=False, threshold=50):
    real_img = real_img.cpu().data.numpy().reshape(img_size, img_size) * 255
    generated_img = generated_img.cpu().data.numpy().reshape(img_size, img_size) * 255
    
    negative = np.zeros_like(real_img)

    if not reverse:
        diff_img = real_img - generated_img
    else:
        diff_img = generated_img - real_img

    diff_img[diff_img <= threshold] = 0

    # 분율 추출
    diff_cnts.append(count_el_not_0(diff_img))
    # 분산 추출
    diff_points.append(np.where(diff_img > threshold))
    
    anomaly_img = np.zeros(shape=(img_size, img_size, 3))
    anomaly_img[:, :, 0] = real_img - diff_img
    anomaly_img[:, :, 1] = real_img - diff_img
    anomaly_img[:, :, 2] = real_img - diff_img
    anomaly_img[:, :, 0] = anomaly_img[:,:,0] + diff_img
    anomaly_img = anomaly_img.astype(np.uint8)

    # anomaly_img 추출
    anomaly_imgs.append(anomaly_img)
    
    fig, plots = plt.subplots(1, 4)

    fig.suptitle(f'Anomaly - (anomaly score: {score:.4})')
    
    fig.set_figwidth(9)
    fig.set_tight_layout(True)
    plots = plots.reshape(-1)
    plots[0].imshow(real_img, cmap='gray', label = "real")
    plots[1].imshow(generated_img, cmap='gray')
    plots[2].imshow(diff_img, cmap='gray')
    plots[3].imshow(anomaly_img)
    
    plots[0].set_title('real')
    plots[1].set_title('generated')
    plots[2].set_title('difference')
    plots[3].set_title('Anomaly Detection')
    plt.show()

# Test

In [ ]:
diff_cnts = []
diff_points = []
anomaly_imgs = []

ano_criterion = nn.MSELoss()
DC.eval()
D.eval()
EC.eval()

# with open("score.csv", "w") as f:
#         f.write("label,img_distance,anomaly_score,z_distance\n")

for i, data in enumerate(test_data_loader, 0):
    real_img = data[0].to(device)

    real_z = EC(real_img) # 진짜 이미지의 latent vector
    fake_img = DC(real_z) # DC에 넣어서 가짜 이미지 생성.
    fake_z = EC(fake_img) # torch.Size([1, 100]) --> latent 진짜 이미지와 매핑된 가짜 이미지의 latent vector

    _, real_feature = D.forward(real_img) # 1, 256
    _, fake_feature = D.forward(fake_img)

    img_distance = ano_criterion(fake_img, real_img)
    loss_feature = ano_criterion(fake_feature, real_feature)

    anomaly_score = img_distance + kappa*loss_feature

    z_distance = ano_criterion(fake_z, real_z)
    
#     with open("score.csv", "a") as f:
#             f.write(f"{label.item()},{img_distance},"
#                     f"{anomaly_score},{z_distance}\n")
            
#     print(f"{label.item()}, {img_distance}, "
#           f"{anomaly_score}, {z_distance}\n")
    compare_imgs(real_img, fake_img, i, anomaly_score, reverse = False, threshold = 0.3)

---

# 분율 추출

In [ ]:
diff_cnts = np.array(diff_cnts)

diff_fraction = diff_cnts / img_size ** 2

print(diff_fraction)

In [ ]:
sum(diff_fraction)/len(diff_fraction)

---

# 분산 추출

In [ ]:
from scipy import stats
from extended_int import int_inf

corr_coeffis = []
corr_p_vals = []

def cal_corr_coeffis():
    for idx in range(len(test_data_loader)):
        x_points = diff_points[idx][0]
        y_points = diff_points[idx][1]
        
        
        if len(x_points) > 0:
            corr_coeffi, corr_p_val = stats.pearsonr(x_points, y_points)
        else:
            corr_coeffi, corr_p_val = -int_inf, -int_inf
        
        corr_coeffis.append(corr_coeffi)
        corr_p_vals.append(corr_p_val)

In [ ]:
cal_corr_coeffis()

print(corr_coeffis)

---

# 모델 저장 및 읽기

In [ ]:
save_file = "./pretrained/pretrained.pth"

def save_pretrained():
    pretrained = {
        "D" : D.state_dict(),
        "DC" : DC.state_dict(),
        "EC" : EC.state_dict()
    }

    if not os.path.isdir("pretrained"):
        os.mkdir("pretrained")
    torch.save(pretrained, save_file)

In [ ]:
save_pretrained()

In [ ]:
pretrained_D = Discriminator().to(device)
pretrained_DC = Decoder().to(device)
pretrained_EC = Encoder().to(device)

def load_pretrained():
    global pretrained_D
    global pretrained_DC
    global pretrained_EC
    
    assert os.path.isdir("pretrained"), "Error : no pretrained dir found!"
    
    pretrained = torch.load(save_file)
    
    pretrained_D.load_state_dict(pretrained["D"])
    pretrained_DC.load_state_dict(pretrained["DC"])
    pretrained_EC.load_state_dict(pretrained["EC"])
    
    #print("pretrained_D :", pretrained_D)
    #print("pretrained_G :", pretrained_DC)
    #print("pretrained_E :", pretrained_EC)

In [ ]:
load_pretrained()

In [ ]:
def test():
    ano_criterion = nn.MSELoss()
    pretrained_DC.eval()
    pretrained_D.eval()
    pretrained_EC.eval()

    # with open("score.csv", "w") as f:
    #         f.write("label,img_distance,anomaly_score,z_distance\n")

    for i, data in enumerate(test_data_loader, 0):
        real_img = data[0].to(device)

        real_z = pretrained_EC(real_img) # 진짜 이미지의 latent vector
        fake_img = pretrained_DC(real_z) # DC에 넣어서 가짜 이미지 생성.
        fake_z = pretrained_EC(fake_img) # torch.Size([1, 100]) --> latent 진짜 이미지와 매핑된 가짜 이미지의 latent vector

        _, real_feature = pretrained_D.forward(real_img) # 1, 256
        _, fake_feature = pretrained_D.forward(fake_img)

        img_distance = ano_criterion(fake_img, real_img)
        loss_feature = ano_criterion(fake_feature, real_feature)

        anomaly_score = img_distance + kappa*loss_feature

        z_distance = ano_criterion(fake_z, real_z)

    #     with open("score.csv", "a") as f:
    #             f.write(f"{label.item()},{img_distance},"
    #                     f"{anomaly_score},{z_distance}\n")

    #     print(f"{label.item()}, {img_distance}, "
    #           f"{anomaly_score}, {z_distance}\n")
        compare_imgs(real_img, fake_img, i, anomaly_score, reverse = False, threshold = 0.3)

In [ ]:
diff_cnts = []
diff_points = []
anomaly_imgs = []

corr_coeffis = []
corr_p_vals = []

test()

In [ ]:
diff_cnts = np.array(diff_cnts)

diff_fraction = diff_cnts / img_size ** 2

print(diff_fraction)

In [ ]:
cal_corr_coeffis()

print(corr_coeffis)

---

# anomaly detection 이미지 추출

In [ ]:
anomaly_imgs

# 이미지 저장 함수

In [ ]:
import cv2

def save_imgs(folder, imgs):
    if not os.path.isdir("anomaly_imgs"):
        os.mkdir("anomaly_imgs")
    
    for i in range(len(imgs)):
        cv2.imwrite('%s/%d.png' %(folder,i), imgs[i]) #이미지 저장할 경로 설정을 여기서 한다.
    print("image saving complete")

In [ ]:
save_imgs("./anomaly_imgs", anomaly_imgs)

---

# 티타늄(Ti64) 상대 밀도 계산

In [ ]:
Ti64_density = 4.43

Ti64_rel_densitys = np.array([])

Ti64_rel_densitys = diff_fraction * Ti64_density

print(Ti64_rel_densitys)